<a href="https://colab.research.google.com/github/newbbo/DeepLearningZeroToAll/blob/master/mldl_work/sentence_data_db_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymysql

In [3]:
import pymysql
import time

def make_conn():
    host = HOST
    port = 3306
    database = DBNAME
    username = USERNAME
    password = PASSWORD

    conn = pymysql.connect(host= host, user= username, password=password, db=database, port=int(port),
                        cursorclass=pymysql.cursors.DictCursor)
    cursor = conn.cursor()

    return cursor

In [4]:
# my_tag = 612

def get_test_sent_list():
    q_gram_sent_id = '''select cast(grammar_tbl.id as char) as gram_id, 
                        grammar_tbl.name as gram_name, 
                        cast(sent_gram.sentence_id as char) as sent_ids
                        from grammar_tbl
                        inner join sentence_grammar_relation_tbl as sent_gram on sent_gram.grammar_id = grammar_tbl.id
                        inner join sentence_tbl as sent on sent.id = sent_gram.sentence_id
                        where 
                        grammar_tbl.id = %s
                        and sent.deleted_at is null
                        and sent.structure is not null
                        '''

    cursor.execute(q_gram_sent_id, [my_tag])
    gram_sent_id= cursor.fetchall()
    gram_sent_ids = set()
    for i in range (0, len(gram_sent_id)):
        sent_ids = gram_sent_id[i]['sent_ids']
        gram_sent_ids.add(sent_ids)
    gram_sent_ids = list(gram_sent_ids)
    return gram_sent_ids[0:2]


def get_pieces_from_tag(gram_sent_ids):
    kor_piece = []
    eng_piece = []
    gram_tags = []
    label_pieces = []
    pos_pieces = []
    kidx_pieces = [] 
    htkidx_pieces = []
    # print(gram_sent_ids)
    
    for i in range (0, len(gram_sent_ids)):
        sentence_id = gram_sent_ids[i]
        my_gram_id = str(my_tag)

        korean_piece, english_piece, gram_tag, label_piece, pos_piece = gram_tag_by_chunck(sentence_id, my_gram_id)

        for k in range (0, len(korean_piece)):
            if len(gram_tag[k]) == 0: 
                pass
            elif list(gram_tag[k])[0]!= my_gram_id:
                pass
            else:
                kor_piece.append(korean_piece[k])
                eng_piece.append(english_piece[k])
                this_gram_tag = re.sub('[^0-9]', '', str(gram_tag[k]))
                gram_tags.append(this_gram_tag)
                label_pieces.append(label_piece[k])
                pos_pieces.append(pos_piece[k])
                kidx_pieces.append(tkidx_piece[k])
                htkidx_pieces.append(htkidx_piece[k])

        if i % 500 == 0:
            print('---- {} 번째 완료 / 총 {} 개')

    chunk_gram_file = pd.DataFrame({'eng_piece' : [], 'kor_piece': [] , 'gram_tags' : []}) # , 'gram_new_label' : []})
    chunk_gram_file['kor_piece'] = kor_piece
    chunk_gram_file['eng_piece'] = eng_piece
    chunk_gram_file['gram_tags'] = gram_tags
    chunk_gram_file['label_pieces'] = label_pieces
    chunk_gram_file['pos_pieces'] = pos_pieces
    chunk_gram_file['kidx_pieces'] = kidx_pieces
    chunk_gram_file['htkidx_pieces'] = htkidx_pieces


    chunk_gram_file.to_csv('test.csv')
    print('----- 파일 생성 완료 -----')

In [5]:
import re

cursor = make_conn()

# sentence_id = '147384335917'

sent_gram_slash_info = dict()
# sent_gram_slash_info['sentence_id'] = sentence_id

##################################################################################################
def get_sentence_word_count(sentence_id):
    q_sentence_word_count = '''
                        select JSON_LENGTH(sent.structure, '$.words') as word_cnt
                        from sentence_tbl as sent
                        where sent.id= %s'''
    cursor.execute(q_sentence_word_count, [sentence_id])
    word_cnt = cursor.fetchall()
    word_cnt = word_cnt[0]['word_cnt']
    if word_cnt == None:
        word_cnt = 0
    return word_cnt

def get_sentence_slash_count(sentence_id):
    q_sentence_slash_count = '''
                        select JSON_LENGTH(sent.structure, '$.slashList') as slash_cnt
                        from sentence_tbl as sent
                        where sent.id= %s'''
    cursor.execute(q_sentence_slash_count, [sentence_id])
    slash_cnt = cursor.fetchall()
    slash_cnt = slash_cnt[0]['slash_cnt']
    if slash_cnt == None:
        slash_cnt = 0 
    return slash_cnt
##################################################################################################

# i 번째 단어에 대한 뜻과 문법태그 가져오기 
def get_grammarid_for_each_word(word_cnt, sentence_id): 
    q_word_info = '''
                        select JSON_EXTRACT(sent.structure, '$.words[%s].word') as word, 
                        JSON_EXTRACT(sent.structure, '$.words[%s].items[*].meanings') as meanings,
                        JSON_EXTRACT(sent.structure, '$.words[%s].grammar[*].grammarId') as gram_id,
                        JSON_EXTRACT(sent.structure, '$.words[%s].label') as label,
                        JSON_EXTRACT(sent.structure, '$.words[%s].partOfSpeech') as pos,
                        JSON_EXTRACT(sent.structure, '$.words[%s].tokenIndex') as tokenIndex,
                        JSON_EXTRACT(sent.structure, '$.words[%s].headTokenIndex') as headTokenIndex
                        from sentence_tbl as sent
                        where sent.id= %s
                        '''
    
    all_word_set = []
    
    for i in range(0, int(word_cnt)):
        cursor.execute(q_word_info, [i, i, i, i, i, i, i, sentence_id])
        word_info = cursor.fetchall()
        word_set = {'word': 'none' , 'meanings' : 'none', 'gram_id' : 'none', 'label' : 'none', 'pos' : 'none',
                   'tokenIndex' : 'none', 'headTokenIndex' : 'none'}
        # word_set = dict()
        word = word_info[0]['word']
        word_meanings = word_info[0]['meanings']
        word_grammarid = word_info[0]['gram_id']
        word_label = word_info[0]['label']
        word_pos = word_info[0]['pos']
        word_tokenIndex = word_info[0]['tokenIndex']
        word_headtokenIndex = word_info[0]['headTokenIndex']
        
        
        word_set['word'] = word
        word_set['meanings'] = word_meanings
        word_set['gram_id'] = word_grammarid
        word_set['label'] = word_label
        word_set['pos'] = word_pos
        word_set['tokenIndex'] = word_tokenIndex
        word_set['headTokenIndex'] = word_headtokenIndex

        all_word_set.append(word_set)
    # print(all_word_set)
    return all_word_set

def get_slash_info(slash_cnt, sentence_id):
    q_slash = '''
                        select JSON_EXTRACT(sent.structure, '$.slashList[%s].start') as start,
                        JSON_EXTRACT(sent.structure, '$.slashList[%s].end') as end,
                        JSON_EXTRACT(sent.structure, '$.slashList[%s].translation') as translation
                        from sentence_tbl as sent
                        where sent.id= %s'''
    slashes_info = []

    for i in range (0, int(slash_cnt)):
        each_slash = {'start': 'none' , 'end' : 'none', 'translation' : 'none'}
        cursor.execute(q_slash, [i, i, i, sentence_id])
        slash_info = cursor.fetchall()

        each_slash['start'] = slash_info[0]['start']
        each_slash['end'] = slash_info[0]['end']
        each_slash['translation'] = slash_info[0]['translation']
        slashes_info.append(each_slash)

    # print(slashes_info)
    return slashes_info

# 문장에 들어있는 전체 문법 태그 가져오기 
def get_grammarid_for_each_sentence(word_cnt, sentence_id):
    q_grammar_id = '''
                        select JSON_EXTRACT(sent.structure, '$.words[%s].grammar[0].grammarId') as gram_id
                        from sentence_tbl as sent
                        where sent.id= %s '''
    gram_id_set = []

    for i in range(0, int(word_cnt)):
        cursor.execute(q_grammar_id, [i, sentence_id])
        gram_id= cursor.fetchall()
        gram_id = gram_id[0]['gram_id']
        if gram_id != None:
            gram_id_set.append(gram_id)
    print(gram_id_set)
    return  gram_id_set
##################################################################################################

def get_slash_all_info(word_cnt, slash_cnt, sentence_id):
    word_info = get_grammarid_for_each_word(word_cnt, sentence_id)
    slash_info = get_slash_info(slash_cnt, sentence_id)

    for i in range (0, len(slash_info)):
        start_idx = int(slash_info[i]['start'])
        end_idx = int(slash_info[i]['end'])
        
        in_slash_list = []

        for j in range(start_idx, end_idx+1):
            in_slash_dict = {'word': 'none' , 'meanings' : 'none', 'gram_id' : 'none'}
            in_slash_dict['word'] = word_info[j]['word']
            in_slash_dict['meanings'] = word_info[j]['meanings']
            in_slash_dict['gram_id'] = word_info[j]['gram_id']
            in_slash_dict['label'] = word_info[j]['label']
            in_slash_dict['pos'] = word_info[j]['pos']
            in_slash_dict['tokenIndex'] = word_info[j]['tokenIndex']
            in_slash_dict['headTokenIndex'] = word_info[j]['headTokenIndex']
            
            in_slash_list.append(in_slash_dict)
        slash_info[i]['in_slash'] = in_slash_list
    # print(slash_info)
    return slash_info

##################################################################################################

def upgrade_meaning(sentence_id):
    word_cnt = get_sentence_word_count(sentence_id)
    slash_cnt = get_sentence_slash_count(sentence_id)

    ans_string = ''
    slash_info = get_slash_all_info(word_cnt, slash_cnt)
    for i in range (0, len(slash_info)):
        ans_string += slash_info[i]['translation']
        for k in range (0, len(slash_info[i]['in_slash'])):
            tmp = str(slash_info[i]['in_slash'][k]['meanings'])
            ans_string += re.sub('[^가-힣]', '', tmp)
    # print(ans_string)
    return ans_string 


#################################################################################################
# 문장 아이디, 영어 청크, 한글 청크, 문법 태그 생성

def gram_tag_by_chunck(sentence_id, my_gram_id):
    word_cnt = get_sentence_word_count(sentence_id)
    slash_cnt = get_sentence_slash_count(sentence_id)

    if word_cnt == 0 or slash_cnt == 0:
        return [], [], []
    else:
        slash_info = get_slash_all_info(word_cnt, slash_cnt, sentence_id)
        english_piece  = []
        korean_piece = []
        gram_tag = []
        label_piece = []
        pos_piece = []
        tkidx_piece = []
        htkidx_piece = []

        english_ans = ''
        label_ans = ''
        pos_ans = ''
        tkidx_ans = ''
        htkid_ans = ''
        my_gram_id = my_gram_id

        for i in range (0, len(slash_info)):
            korean_ans = str(slash_info[i]['translation'])
            korean_ans = re.sub('[^a-zA-Z0-9가-힇 ]', '', korean_ans)
            gram_tmp = []
            for k in range (0, len(slash_info[i]['in_slash'])):
                english_tmp = str(slash_info[i]['in_slash'][k]['word'])
                english_tmp = re.sub('[^a-zA-Z0-9 ]', '', english_tmp)
                
                gram_id_tmp = slash_info[i]['in_slash'][k]['gram_id']
                english_ans += english_tmp
                english_ans += ' '
                if gram_id_tmp is None:
                    pass
                elif my_gram_id in gram_id_tmp:
                    gram_tmp.append(my_gram_id)
                label_tmp = str(slash_info[i]['in_slash'][k]['label'])
                label_ans += label_tmp
                label_ans += ' '
                pos_tmp = str(slash_info[i]['in_slash'][k]['pos'])
                pos_ans += pos_tmp
                pos_ans += ' '
                tkidx_tmp = str(slash_info[i]['in_slash'][k]['tokenIndex'])
                tkidx_ans += tkidx_tmp
                tkidx_ans += ' '
                htkidx_tmp = str(slash_info[i]['in_slash'][k]['headtokenIndex'])
                htkid_ans += htkidx_tmp
                htkid_ans += ' '
                       
            gram_tag.append(set(gram_tmp))
            korean_piece.append(korean_ans)
            english_piece.append(english_ans)
            label_piece.append(label_ans)
            pos_piece.append(pos_ans)
            tkidx_piece.append(tkidx_ans)
            htkidx_piece.append(htkid_ans)
            english_ans = ''
            label_ans = ''
            pos_ans = ''
            tkidx_ans = ''
            htkid_ans = ''
            

        return korean_piece, english_piece, gram_tag, label_piece, pos_piece , tkidx_piece, htkidx_piece
    
    
######
#################################################################################################
# slash_info의 정보를 열 별로 저장 

def chunck_to_dataframe(slash_info):
    if word_cnt == 0 or slash_cnt == 0:
        return [], [], []
    else:
        english_piece  = []
        korean_piece = []
        gram_tag = []
        label_piece = []
        pos_piece = []
        pos_label_piece = []
        tkidx_piece = []
        htkidx_piece = []

        english_ans = ''
        label_ans = ''
        pos_ans = ''
        pos_label_ans = ''
        tkidx_ans = ''
        htkid_ans = ''
        my_gram_id = ''

        for i in range (0, len(slash_info)):
            korean_ans = str(slash_info[i]['translation'])
            korean_ans = re.sub('[^a-zA-Z0-9가-힇 ]', '', korean_ans)
            gram_tmp = ''
            for k in range (0, len(slash_info[i]['in_slash'])):
                english_tmp = str(slash_info[i]['in_slash'][k]['word'])
                english_tmp = re.sub('[^a-zA-Z0-9 ]', '', english_tmp)
                english_ans += english_tmp
                
                gram_id_tmp = slash_info[i]['in_slash'][k]['gram_id']
                if gram_id_tmp is None:
                    pass
                else:
                    gram_tmp += gram_id_tmp
                                
                # gram_tmp.append(gram_id_tmp)
                label_tmp = re.sub('[^a-zA-Z0-9 ]', '', str(slash_info[i]['in_slash'][k]['label']))
                label_ans += label_tmp
                
            
                pos_tmp = re.sub('[^a-zA-Z0-9 ]', '', str(slash_info[i]['in_slash'][k]['pos']))
                pos_ans += pos_tmp
                
                pos_label_ans += pos_tmp + ' '
                pos_label_ans += label_tmp + ' '
                
                tkidx_tmp = str(slash_info[i]['in_slash'][k]['tokenIndex'])
                tkidx_ans += tkidx_tmp
                
                
                htkidx_tmp = str(slash_info[i]['in_slash'][k]['headTokenIndex'])
                htkid_ans += htkidx_tmp
                
                if k != len(slash_info[i]['in_slash'])-1:
                    english_ans += ' '
                    label_ans += ' '
                    pos_ans += ' '
                    gram_tmp += ' '
                    pos_label_ans += ' '
                    tkidx_ans += ' '
                    htkid_ans += ' '
                    
                else:
                    pass
                
              
            gram_tag.append(gram_tmp)
            korean_piece.append(korean_ans)
            english_piece.append(english_ans)
            label_piece.append(label_ans)
            pos_piece.append(pos_ans)
            pos_label_piece.append(pos_label_ans)
            tkidx_piece.append(tkidx_ans)
            #print("tkidx_ans : ", tkidx_ans)
            htkidx_piece.append(htkid_ans)
            #print("htkidx_ans : ", htkid_ans)
            english_ans = ''
            label_ans = ''
            pos_ans = ''
            pos_label_ans = ''
            tkidx_ans = ''
            htkid_ans = ''
            
        #print(tkidx_piece)
        return korean_piece, english_piece, gram_tag, label_piece, pos_piece, pos_label_piece, tkidx_piece, htkidx_piece
      

In [6]:
def select_all_sent_ids():
    q_sent_id = '''select id as sent_id
                    from sentence_tbl
                    where sentence_tbl.structure is not null'''

    cursor.execute(q_sent_id)
    sent_id= cursor.fetchall()
    all_sent_ids = set()
    for i in range (0, len(sent_id)):
        sent_ids = sent_id[i]['sent_id']
        all_sent_ids.add(sent_ids)
    all_sent_ids = list(all_sent_ids)
    return all_sent_ids

In [7]:
def make_re_slash_final_gram(gram_tag):

    slash_final_gram = []
    for item in gram_tag:
        #item = re.sub('[^0-9]', '', item)
        #print(item)
        slash_final_gram.append(item.split(' '))
    #print(">>>", slash_final_gram)
    re_slash_final_gram = []
    for item in slash_final_gram:
        #print("###", set(item))
        sample_list = [re.sub('[^0-9]', '', v) for v in item if v]
        #print(set(sample_list))
        re_slash_final_gram.append(set(sample_list))
    
    return re_slash_final_gram

In [8]:
sentence_id = 26660045280
word_cnt = get_sentence_word_count(sentence_id)
slash_cnt = get_sentence_slash_count(sentence_id)
slash_info = get_slash_all_info(word_cnt, slash_cnt, sentence_id)
print(slash_cnt)

0


In [9]:
sentence_id = 79695568212
word_cnt = get_sentence_word_count(sentence_id)
slash_cnt = get_sentence_slash_count(sentence_id)
slash_info = get_slash_all_info(word_cnt, slash_cnt, sentence_id)
print(slash_info)
if slash_cnt == 0:
    print('no_slash')

[{'start': '0', 'end': '5', 'translation': '"당신이 패스트푸드를 먹을 때"', 'in_slash': [{'word': '"When"', 'meanings': '[["~할 때"], []]', 'gram_id': '[1020]', 'label': '"ADVMOD"', 'pos': '"ADV"', 'tokenIndex': '7', 'headTokenIndex': '9'}, {'word': '"you"', 'meanings': '[[], []]', 'gram_id': None, 'label': '"NSUBJ"', 'pos': '"PRON"', 'tokenIndex': '8', 'headTokenIndex': '9'}, {'word': '"eat"', 'meanings': '[["먹다"], ["음식", "식사"]]', 'gram_id': '[201]', 'label': '"ADVCL"', 'pos': '"VERB"', 'tokenIndex': '9', 'headTokenIndex': '26'}, {'word': '"fast"', 'meanings': '[["단식하다"], [], ["빠른", " 신속한"], ["빨리"]]', 'gram_id': None, 'label': '"AMOD"', 'pos': '"ADJ"', 'tokenIndex': '10', 'headTokenIndex': '11'}, {'word': '"food"', 'meanings': '[["음식"]]', 'gram_id': None, 'label': '"DOBJ"', 'pos': '"NOUN"', 'tokenIndex': '11', 'headTokenIndex': '9'}, {'word': '","', 'meanings': None, 'gram_id': None, 'label': '"P"', 'pos': '"PUNCT"', 'tokenIndex': '12', 'headTokenIndex': '11'}]}, {'start': '6', 'end': '15', 'transl

In [10]:
print(slash_info[0]['in_slash'][0]['word'])
print(slash_info[0]['in_slash'][0]['pos'])
print(slash_info[0]['in_slash'][0]['headTokenIndex'])
head = slash_info[0]['in_slash'][0]['headTokenIndex']

"When"
"ADV"
9


In [ ]:
#################################

In [ ]:
import pandas as pd
from tqdm import tqdm

sent_list = select_all_sent_ids()
print(">>>>........ 문장 개수 : ", len(sent_list))

>>>>........ 문장 개수 :  756142


In [ ]:
len(set(sent_list))

# 615013
# 6/14 : 756142

756142

In [ ]:
chunk_data = pd.DataFrame({})

sent_id_add = []
korean_piece_add = []
english_piece_add = [] 
gram_tag_add = [] 
label_piece_add = [] 
pos_piece_add = []
pos_label_piece_add = []
thidx_add = []
htkidx_add = []

#sent_list = ['79695568212']

for sent_id in tqdm(sent_list):
    # print(sent_id)
    word_cnt = get_sentence_word_count(sent_id)
    slash_cnt = get_sentence_slash_count(sent_id)
    slash_info = get_slash_all_info(word_cnt, slash_cnt, sent_id)
    if slash_cnt == 0:
        pass
    else:
        korean_piece, english_piece, gram_tag, label_piece, pos_piece, pos_label_piece, tkidx_piece, htkidx_piece = chunck_to_dataframe(slash_info)
        #print(htkidx_piece)
        final_chunck_gram = make_re_slash_final_gram(gram_tag)
        
        dep_str = ''
    
        for i in range (0, len(korean_piece)):
            sent_id_add.append(sent_id)
            korean_piece_add.append(korean_piece[i]) 
            english_piece_add.append(english_piece[i]) 
            gram_tag_add.append(final_chunck_gram[i])
            label_piece_add.append(label_piece[i])
            pos_piece_add.append(pos_piece[i])
            pos_label_piece_add.append(pos_label_piece[i])
            thidx_add.append(tkidx_piece[i])
            htkidx_add.append(htkidx_piece[i])
        
            
chunk_data['sent_id'] = sent_id_add
chunk_data['english_piece'] = english_piece_add
chunk_data['korean_piece'] = korean_piece_add
chunk_data['gram_tag'] = gram_tag_add
chunk_data['pos_piece'] = pos_piece_add
chunk_data['label_piece'] = label_piece_add
chunk_data['pos_label_piece'] = pos_label_piece_add
chunk_data['thidx'] = thidx_add
chunk_data['htkidx'] = htkidx_add


chunk_data.to_csv('all_chunk_data+headidx.csv')
# chunk_data.to_csv('head_test.csv')
#chunk_data.to_csv('all_chunk_data+head.csv')

 33%|███▎      | 248917/756142 [2:08:45<4:25:37, 31.83it/s] 